# Imports 

In [189]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader as dl
import torchvision 
import torchvision.transforms as transforms

from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from skimage import io

import os, shutil
import urllib.request
import zipfile
import tarfile

In [199]:
sys.path

['../../Utils/',
 '../../Utils/',
 '',
 '/usr/local/lib/python36.zip',
 '/usr/local/lib/python3.6',
 '/usr/local/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/site-packages/xgboost-0.72-py3.6.egg',
 '/usr/local/lib/python3.6/site-packages/IPython/extensions',
 '/home/mvisaya/.ipython']

In [201]:
sys.path.remove('../../Utils/')

In [202]:
sys.path

['',
 '/usr/local/lib/python36.zip',
 '/usr/local/lib/python3.6',
 '/usr/local/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/site-packages/xgboost-0.72-py3.6.egg',
 '/usr/local/lib/python3.6/site-packages/IPython/extensions',
 '/home/mvisaya/.ipython']

In [188]:
sys.path


['',
 '/usr/local/lib/python36.zip',
 '/usr/local/lib/python3.6',
 '/usr/local/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/site-packages/xgboost-0.72-py3.6.egg',
 '/usr/local/lib/python3.6/site-packages/IPython/extensions',
 '/home/mvisaya/.ipython']

In [87]:
def get_lfw(datasets_dir): 

    if os.path.isdir(os.path.join(datasets_dir,'lfw')):
        print('LFW already downloaded.')
        return

    if not os.path.isdir(datasets_dir):
        os.makedirs(datasets_dir)

    print('Downloading LFW.')

    url = 'http://vis-www.cs.umass.edu/lfw/lfw.tgz'
    urllib.request.urlretrieve(url, os.path.join(datasets_dir,'lfw.tgz'))

    tar = tarfile.open(os.path.join(datasets_dir,'lfw.tgz'))
    tar.extractall(path=os.path.join(datasets_dir,'lfw/'))

    os.rename(os.path.join(datasets_dir,'lfw/lfw/'), os.path.join(datasets_dir,'lfw/lfw_original/'))


    lfw_dir = os.path.join(datasets_dir,'lfw/lfw_original/')
    people_dir = os.listdir(lfw_dir)


    num_per_class = 20

    new_dir = os.path.join(datasets_dir,'lfw/lfw_'+str(num_per_class))

    if not os.path.isdir(new_dir):
        os.makedirs(new_dir)

    for p in people_dir:
        imgs = os.listdir(os.path.join(lfw_dir,p))
        if len(imgs) >= num_per_class:
            shutil.copytree(os.path.join(lfw_dir,p),os.path.join(new_dir,p))

    print('LFW successfully downloaded and preprocessed.')

In [88]:
get_lfw('./data')
data_dir = "./data/lfw/lfw_20/"

LFW already downloaded.


In [126]:
images=[] #unbalanced dataset
eqimages=[]; #balanced but small dataset
imgs_per_class=np.zeros(len(classes))
for i in os.listdir(data_dir):
    for j in os.listdir(os.path.join(data_dir,i)):
        images.append(os.path.join(data_dir,i,j))
        
classes=[]
classes_to_idx={}
j=0;


for i in images:
    name=i.split('/')[4];
    if name not in classes_to_idx:
        classes.append(name)
        classes_to_idx[name]=j
        j+=1

for k in images:
    name=k.split('/')[4];
    name_idx=classes_to_idx[name]
    if imgs_per_class[name_idx]<20:
        eqimages.append(k)
        imgs_per_class[name_idx]+=1



In [141]:
images=np.random.permutation(images)
eqimages=np.random.permutation(eqimages)
trainset_len=.8*(len(images))
trainset=images[:trainset_len];
testset=images[trainset_len:]
print(len(trainset));print(len(testset))

TypeError: slice indices must be integers or None or have an __index__ method

In [131]:
class LFW(torch.utils.data.Dataset):
    def __init__(self, img_list, classes_list, transform=None):
        self.classes_to_idx=classes_list
        self.img_list=img_list
        self.transform=transform;
    
    def __len__(self):
        return len(self.img_list);
    
    def __getitem__(self,idx):
        img=self.img_list[idx]
        sample=io.imread(img)
        label=self.classes_to_idx[img.split('/')[4]]
        if self.transform is not None:
            sample=self.transform(sample)
        return sample, label


In [132]:
transform=transforms.ToTensor()
trainset=LFW(eqimages,classes_to_idx,transform=transform)
testset=LFW(images,classes_to_idx,transform=transform)

In [133]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=trainset.__len__(),
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=150,shuffle=True,num_workers=2)

In [135]:
def flatten(a):
    y=a.view(a.shape[0],-1)
    return y
def accuracy(preds,targs,training=False):
    t=torch.from_numpy(preds)
    correct = (torch.eq(t,targs)).sum().item()
    accuracy=100*(correct/len(preds))
    sys.stdout.write("\033[1;31m")
    if training is False:
        print('overall testing accuracy: ',accuracy)
    else:
        print('overall training accuracy: ',accuracy)
    sys.stdout.write("\033[0;0m")    
    return accuracy

In [136]:
components=180
pca=PCA(n_components=components)
scaling = MinMaxScaler(feature_range=(-1,1))


In [137]:
classifiers = []
C_range = np.logspace(-2,2,5) #[1.e-02 1.e-01 1.e+00 1.e+01 1.e+02]
gamma_range = np.logspace(-3, 1, 5) #[1.e-03 1.e-02 1.e-01 1.e+00 1.e+01]

for c in C_range:
    for gamma in gamma_range:
        svcrbf=svm.SVC(C=c,kernel='rbf',gamma=gamma)
        svc=make_pipeline(pca,scaling,svcrbf)
        classifiers.append([gamma,c,svc])

In [140]:
train_accuracies=[]
classif_idx=0;
for j in range(10):
    for data in trainloader:
        traininputs,traintargets=data;
        traininputs=flatten(traininputs)
        classifiers[classif_idx][2]=classifiers[classif_idx][2].fit(traininputs,traintargets)
        train_accuracies.append(accuracy(classifiers[classif_idx][2].predict(traininputs),traintargets,training=True))
        classif_idx+=1

NameError: name 'trainaccuracies' is not defined

In [ ]:
testing_accuracies=[]
for data in testloader:
    testinputs, testtargets=data;
    testinputs=flatten(testinputs)
for i in classifiers:
    x=accuracy(i[2].predict(testinputs),testtargets)
    testing_accuracies.append(x)

